<a href="https://colab.research.google.com/github/tylerlum/traffic_signs/blob/master/Traffic_Signs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## Get traffic sign data
!git clone https://bitbucket.org/jadslim/german-traffic-signs
!ls german-traffic-signs

In [0]:
import numpy as np
import matplotlib.pyplot as plt

## Keras
import keras 
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

## Pickle and pandas
import pickle
import pandas as pd

## Random
import random
np.random.seed(0)

In [0]:
## Unpickle files
with open('german-traffic-signs/train.p', 'rb') as f:
  train_data = pickle.load(f)
with open('german-traffic-signs/valid.p', 'rb') as f:
  val_data = pickle.load(f)
with open('german-traffic-signs/test.p', 'rb') as f:
  test_data = pickle.load(f)
  
print(type(train_data))

## Get data and labels
X_train, Y_train = train_data['features'], train_data['labels']
X_val, Y_val = val_data['features'], val_data['labels']
X_test, Y_test = test_data['features'], test_data['labels']

## Check shape (number of pictures, width, height, RGB)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [0]:
## Check for bad data
assert(X_train.shape[0] == Y_train.shape[0]), "The number of images is not equal to the number of labels"
assert(X_val.shape[0] == Y_val.shape[0]), "The number of images is not equal to the number of labels"
assert(X_test.shape[0] == Y_test.shape[0]), "The number of images is not equal to the number of labels"

assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3"
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3"
assert(X_test.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3"

In [0]:
## Use pandas to import data from CSV file
data = pd.read_csv('german-traffic-signs/signnames.csv')
num_of_samples = []

cols = 5
num_classes = data.shape[0]

fig, axs = plt.subplots(nrows=num_classes, ncols=cols, figsize=(5,50))
fig.tight_layout()

## Iterate over dataframe rows as (index, Series)
for i in range(cols):
  for j, row in data.iterrows():
    x_selected = X_train[Y_train == j]
    axs[j][i].imshow(x_selected[random.randint(0,len(x_selected)-1), :, :], cmap=plt.get_cmap('gray'))
    axs[j][i].axis('off')
    if i == int(cols/2):
      axs[j][i].set_title(str(j) + "-" + row["SignName"])
      num_of_samples.append(len(x_selected))

In [0]:
## Graph our data
print(num_of_samples)
plt.figure(figsize=(12,4))
plt.bar(range(0, num_classes), num_of_samples)
plt.title('Distribution of the train dataset')
plt.xlabel('Class number')
plt.ylabel('Number of images')
plt.show()

In [0]:
## Preprocess images
import cv2

def grayscale(img):
  """Color not reliable because of different lighting. Convert to gray"""
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  return img

def equalize(img):
  """Histogram Equalization (standardize lighting)"""
  img = cv2.equalizeHist(img)  # must be grayscale img
  return img

def preprocessing(img):
  """Grayscale, equalize and normalize"""
  img = grayscale(img)
  img = equalize(img)
  img = img / 255
  return img

X_train = np.array(list(map(preprocessing, X_train)))
X_val = np.array(list(map(preprocessing, X_val)))
X_test = np.array(list(map(preprocessing, X_test)))

In [0]:
## Test preprocessing
plt.imshow(X_train[random.randint(0, len(X_train)-1)])
plt.axis('off')
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [0]:
## Reshape input data to have depth of 1
X_train = X_train.reshape(34799, 32, 32, 1)
X_val = X_val.reshape(4410, 32, 32, 1)
X_test = X_test.reshape(12630, 32, 32, 1)

## One-Hot Encode outputs
Y_train = to_categorical(Y_train)
Y_val = to_categorical(Y_val)
Y_test = to_categorical(Y_test)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [0]:
def leNet_model():
  """LeNet Model"""
  model = Sequential()
  model.add(Conv2D(30, (5,5), input_shape=(32,32,1), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Conv2D(15, (3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dense(500, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))
  
  model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [0]:
## Create leNet model
model = leNet_model()
print(model.summary())
history = model.fit(X_train, Y_train, epochs=10, validation_data=(X_val, Y_val), batch_size=400, verbose=1, shuffle=1)